In [2]:
import pandas as pd  
import numpy as np  
from sklearn.svm import SVR
from sklearn import metrics
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV

In [3]:
RDL = pd.read_excel("/Users/moca_lucy/Documents/大學/專題/RDL_AlCu/PCA_newdata(RDL).xlsx", sheet_name="Weekly", header=0)

In [24]:
traindata = RDL[0:48]
testdata = RDL[48:57]
x_columns = ["PC1", "PC2", "PC3", "PC4"]
x_train = traindata[x_columns]
traindata
y_train = traindata["Y"]
x_test = testdata[x_columns]
y_test = testdata["Y"]
#type(y_train)
testdata

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Y
48,1044.2385,573.4517,-1079.2069,261.9190,1498.0311,-93.7065,2007.9200,2092.6781,-630.3695,120.0764,4
49,1656.4861,-3.9352,-1303.0965,-926.0304,2161.8990,263.1273,2494.4983,2043.0433,-406.7911,110.7683,4
50,1204.0967,2002.7321,-1740.0187,2487.0047,2012.2293,-783.5962,3287.6613,4317.5268,-1628.8761,252.2684,8
51,1763.0929,1352.9464,-1880.2648,649.7470,2149.1095,-466.5297,3509.4190,3902.3773,-1249.3122,222.4029,11
52,2175.1134,-569.8720,-1447.5768,-1681.7323,3187.8063,724.4069,2990.7227,2021.1155,-205.4828,105.1344,3
53,1965.4572,173.8332,-1582.7163,-179.4093,3253.9487,359.9231,3273.3746,2888.7966,-683.3862,158.1895,7
54,918.5448,1595.6277,-1360.1255,1531.7679,1376.7973,-606.8590,2521.8732,3230.7282,-1193.6469,190.5118,5
55,448.2947,459.2179,-599.6670,453.3555,931.8743,-40.8359,1114.5769,1222.3185,-401.6977,76.7181,5
56,725.5582,645.2500,-877.0940,654.7692,1381.1701,-84.5414,1681.1552,1842.3523,-601.2040,107.3553,2


In [18]:
cls = SVR(kernel="rbf", C=23)
cls.fit(x_train, y_train)
print("the test score of SVR:{:.3f}".format(cls.score(x_train, y_train)))
print("the test score of SVR:{:.3f}".format(cls.score(x_test, y_test)))

the test score of SVR:0.479
the test score of SVR:0.542


In [ ]:
'''parameters = {'kernel': ('linear', 'rbf'), 'C': [1, 2, 4], 'gamma': [0.125, 0.25, 0.5, 1, 2, 4]}
clf = GridSearchCV(cls, param_grid=parameters)
grid_search = clf.fit(x_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print(grid_search.best_estimator_)'''

In [ ]:
#gammas = np.logspace(-2, 1)
#gammas

In [ ]:
#svr_predict = cls.predict(x_test)

In [19]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    sum=0
    denom=0
    for index, value in enumerate(y_true):
        if value:
            sum+=np.abs((y_true[index]-y_pred[index])/y_true[index])
            denom+=1
    return (sum/denom)*100

In [20]:
def mase(y_true, y_pred):
    y_num = y_true.size
    d = np.abs(np.diff(y_true)).sum()/(y_num-1)
    ret = np.abs(y_true - y_pred)
    return ret.mean()/d

In [21]:
def table(lm, x, y, w):
    y_pred = lm.predict(x)
    y_true = y
   # print(y_true[190])
    #print(np.where(np.isnan(y_pred)))
    #print(y_pred, y_true)
    MAE = round(metrics.mean_absolute_error(y_true, y_pred), 4)
    #print(metrics.mean_absolute_error(y_true, y_pred))
    MSE = round(metrics.mean_squared_error(y_true, y_pred), 4)
    RMSE = round(np.sqrt(metrics.mean_squared_error(y_true, y_pred)), 4)
    MAPE =  str(round(mape(y_true, y_pred), 4))+'%'
    MASE = round(mase(y_true, y_pred), 4)
    errortype = ["MAE", "MSE", "RMSE", "MAPE", "MASE"]
    error = [MAE, MSE, RMSE, MAPE, MASE]
    #print(error)
    if w == "train":
        error_dict = {" ":errortype, "training":error}
    elif w == "test":
        error_dict = {" ":errortype, "testing":error}
    #error_dict = {" ":errortype, "誤差分析":error}
    pd.options.display.float_format = '{:.4f}'.format
    df = pd.DataFrame(error_dict)
    df = df.set_index(" ").rename_axis(None)
    #print(df)
    return df

df1 = table(cls, x_train, y_train, "train")
df1

,training
MAE,1.1330
MSE,2.6657
RMSE,1.6327
MAPE,24.9253%
MASE,0.4260


In [25]:
df2 = table(cls, x_test, y_test, "test")
df2

,testing
MAE,1.4795
MSE,3.0553
RMSE,1.7479
MAPE,31.0155%
MASE,0.4932
